In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import re
import numpy as np
import pandas as pd
import spacy
import string
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.utils import resample

from imblearn.over_sampling import SMOTE

In [ ]:
import joblib
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('omw-1.4')
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import log_loss, f1_score
from xgboost import XGBClassifier

from sklearn.metrics import accuracy_score

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/data.csv")
df.head()

,Sentence,Sentiment
0,The GeoSolutions technology will leverage Bene...,positive
1,"$ESI on lows, down $1.50 to $2.50 BK a real po...",negative
2,"For the last quarter of 2010 , Componenta 's n...",positive
3,According to the Finnish-Russian Chamber of Co...,neutral
4,The Swedish buyout firm has sold its remaining...,neutral


In [ ]:
df.shape

(5842, 2)

In [ ]:
df['Sentence'][4]

'The Swedish buyout firm has sold its remaining 22.4 percent stake , almost eighteen months after taking the company public in Finland .'

In [ ]:
df['Sentiment'].value_counts()

neutral     3130
positive    1852
negative     860
Name: Sentiment, dtype: int64

In [ ]:
label_encoder=preprocessing.LabelEncoder()
df['Sentiment']=label_encoder.fit_transform(df["Sentiment"])
df[['Sentiment']]

,Sentiment
0,2
1,0
2,2
3,1
4,1
...,...
5837,0
5838,1
5839,1
5840,1


Text Preprocessing

Removal Of URLS

In [ ]:
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

df["text_rv_url"] = df["Sentence"].apply(lambda text: remove_urls(text))
df[['Sentence', 'text_rv_url']].head()

,Sentence,text_rv_url
0,The GeoSolutions technology will leverage Bene...,The GeoSolutions technology will leverage Bene...
1,"$ESI on lows, down $1.50 to $2.50 BK a real po...","$ESI on lows, down $1.50 to $2.50 BK a real po..."
2,"For the last quarter of 2010 , Componenta 's n...","For the last quarter of 2010 , Componenta 's n..."
3,According to the Finnish-Russian Chamber of Co...,According to the Finnish-Russian Chamber of Co...
4,The Swedish buyout firm has sold its remaining...,The Swedish buyout firm has sold its remaining...


Removal Of HTML Tags

In [ ]:
def remove_html(text):
    html_pattern = re.compile('<.*?>')
    return html_pattern.sub(r'', text)

df["text_rv_html"] = df["text_rv_url"].apply(lambda text: remove_html(text))
df[['text_rv_url', 'text_rv_html']].head()

,text_rv_url,text_rv_html
0,The GeoSolutions technology will leverage Bene...,The GeoSolutions technology will leverage Bene...
1,"$ESI on lows, down $1.50 to $2.50 BK a real po...","$ESI on lows, down $1.50 to $2.50 BK a real po..."
2,"For the last quarter of 2010 , Componenta 's n...","For the last quarter of 2010 , Componenta 's n..."
3,According to the Finnish-Russian Chamber of Co...,According to the Finnish-Russian Chamber of Co...
4,The Swedish buyout firm has sold its remaining...,The Swedish buyout firm has sold its remaining...


Convert to LOWERCASE

In [ ]:
df["text_lower"] = df["text_rv_html"].str.lower()
df[['text_rv_html', 'text_lower']].head()

,text_rv_html,text_lower
0,The GeoSolutions technology will leverage Bene...,the geosolutions technology will leverage bene...
1,"$ESI on lows, down $1.50 to $2.50 BK a real po...","$esi on lows, down $1.50 to $2.50 bk a real po..."
2,"For the last quarter of 2010 , Componenta 's n...","for the last quarter of 2010 , componenta 's n..."
3,According to the Finnish-Russian Chamber of Co...,according to the finnish-russian chamber of co...
4,The Swedish buyout firm has sold its remaining...,the swedish buyout firm has sold its remaining...


Removal Of Punctuation

In [ ]:
punctuation = string.punctuation
print(punctuation)

def remove_punctuation(text):
    return text.translate(str.maketrans('', '', punctuation))

df["text_rv_punct"] = df["text_lower"].apply(lambda text: remove_punctuation(text))
df[['text_lower', 'text_rv_punct']].head()

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


,text_lower,text_rv_punct
0,the geosolutions technology will leverage bene...,the geosolutions technology will leverage bene...
1,"$esi on lows, down $1.50 to $2.50 bk a real po...",esi on lows down 150 to 250 bk a real possibility
2,"for the last quarter of 2010 , componenta 's n...",for the last quarter of 2010 componenta s net...
3,according to the finnish-russian chamber of co...,according to the finnishrussian chamber of com...
4,the swedish buyout firm has sold its remaining...,the swedish buyout firm has sold its remaining...


Removal Digits

In [ ]:
from string import digits

def remove_digits(text):
    return text.translate(str.maketrans('', '', digits))

df["text_rv_digits"] = df["text_rv_punct"].apply(lambda text: remove_digits(text))
df[['text_rv_punct', 'text_rv_digits']].head()

,text_rv_punct,text_rv_digits
0,the geosolutions technology will leverage bene...,the geosolutions technology will leverage bene...
1,esi on lows down 150 to 250 bk a real possibility,esi on lows down to bk a real possibility
2,for the last quarter of 2010 componenta s net...,for the last quarter of componenta s net sal...
3,according to the finnishrussian chamber of com...,according to the finnishrussian chamber of com...
4,the swedish buyout firm has sold its remaining...,the swedish buyout firm has sold its remaining...


*LEMMATIZATION*

In [ ]:
from nltk.stem.wordnet import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [ ]:
def lemmatize_words(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])

df["text_lemmatized"] = df["text_rv_digits"].apply(lambda text: lemmatize_words(text))
df[['text_rv_digits','text_lemmatized']].head()

,text_rv_digits,text_lemmatized
0,the geosolutions technology will leverage bene...,the geosolutions technology will leverage bene...
1,esi on lows down to bk a real possibility,esi on low down to bk a real possibility
2,for the last quarter of componenta s net sal...,for the last quarter of componenta s net sale ...
3,according to the finnishrussian chamber of com...,according to the finnishrussian chamber of com...
4,the swedish buyout firm has sold its remaining...,the swedish buyout firm ha sold it remaining p...


TOKENIZATION

In [ ]:
from nltk import sent_tokenize, word_tokenize
var=df['text_lemmatized'].tolist()
list=[]
for i in var:
    tokens_ = nltk.word_tokenize(i)
    list.append(tokens_)
print(list)

[['the', 'geosolutions', 'technology', 'will', 'leverage', 'benefon', 's', 'gps', 'solution', 'by', 'providing', 'location', 'based', 'search', 'technology', 'a', 'community', 'platform', 'location', 'relevant', 'multimedia', 'content', 'and', 'a', 'new', 'and', 'powerful', 'commercial', 'model'], ['esi', 'on', 'low', 'down', 'to', 'bk', 'a', 'real', 'possibility'], ['for', 'the', 'last', 'quarter', 'of', 'componenta', 's', 'net', 'sale', 'doubled', 'to', 'eurm', 'from', 'eurm', 'for', 'the', 'same', 'period', 'a', 'year', 'earlier', 'while', 'it', 'moved', 'to', 'a', 'zero', 'pretax', 'profit', 'from', 'a', 'pretax', 'loss', 'of', 'eurm'], ['according', 'to', 'the', 'finnishrussian', 'chamber', 'of', 'commerce', 'all', 'the', 'major', 'construction', 'company', 'of', 'finland', 'are', 'operating', 'in', 'russia'], ['the', 'swedish', 'buyout', 'firm', 'ha', 'sold', 'it', 'remaining', 'percent', 'stake', 'almost', 'eighteen', 'month', 'after', 'taking', 'the', 'company', 'public', 'in',

TF-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidfModel = TfidfVectorizer()
print(tfidfModel.fit_transform(df['text_lemmatized']))

  (0, 5369)	0.17407921174677116
  (0, 1557)	0.18445495473579346
  (0, 6449)	0.24943058374049829
  (0, 5595)	0.11940521054937077
  (0, 353)	0.12241758206353347
  (0, 1732)	0.1877517080738447
  (0, 5478)	0.24106185670476432
  (0, 6991)	0.22926679841923162
  (0, 6332)	0.20461874005706013
  (0, 1576)	0.21747174013369894
  (0, 7508)	0.22926679841923162
  (0, 711)	0.16180331287585895
  (0, 4866)	0.39147025781183814
  (0, 6648)	0.19424299706803783
  (0, 1152)	0.09783268626077347
  (0, 7903)	0.13784462784215298
  (0, 3463)	0.22926679841923162
  (0, 799)	0.2144067823893045
  (0, 4760)	0.24106185670476432
  (0, 9388)	0.08861692847838103
  (0, 8455)	0.2867168710823768
  (0, 3377)	0.24943058374049829
  (0, 8547)	0.04371714913580061
  (1, 6424)	0.4565473225285709
  (1, 6849)	0.3437318363674069
  :	:
  (5840, 8547)	0.09976654756721565
  (5841, 8170)	0.26817362187974514
  (5841, 4629)	0.2774835636116067
  (5841, 4465)	0.2609522700489871
  (5841, 4014)	0.23262042836219793
  (5841, 6569)	0.213004715207

In [ ]:
print(tfidfModel.fit_transform(df['text_lemmatized']).todense())

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [ ]:
tfidfModel.vocabulary_

{'the': 8547,
 'geosolutions': 3377,
 'technology': 8455,
 'will': 9388,
 'leverage': 4760,
 'benefon': 799,
 'gps': 3463,
 'solution': 7903,
 'by': 1152,
 'providing': 6648,
 'location': 4866,
 'based': 711,
 'search': 7508,
 'community': 1576,
 'platform': 6332,
 'relevant': 6991,
 'multimedia': 5478,
 'content': 1732,
 'and': 353,
 'new': 5595,
 'powerful': 6449,
 'commercial': 1557,
 'model': 5369,
 'esi': 2698,
 'on': 5857,
 'low': 4928,
 'down': 2341,
 'to': 8663,
 'bk': 884,
 'real': 6849,
 'possibility': 6424,
 'for': 3158,
 'last': 4668,
 'quarter': 6729,
 'of': 5808,
 'componenta': 1627,
 'net': 5581,
 'sale': 7368,
 'doubled': 2334,
 'eurm': 2741,
 'from': 3245,
 'same': 7383,
 'period': 6201,
 'year': 9512,
 'earlier': 2423,
 'while': 9362,
 'it': 4261,
 'moved': 5445,
 'zero': 9564,
 'pretax': 6513,
 'profit': 6581,
 'loss': 4915,
 'according': 50,
 'finnishrussian': 3060,
 'chamber': 1344,
 'commerce': 1556,
 'all': 258,
 'major': 5029,
 'construction': 1714,
 'company': 

In [ ]:
import pickle

tfidf_matrix = tfidfModel.fit_transform(df['text_lemmatized']).todense()

# Save the TF-IDF matrix
np.save("/content/drive/MyDrive/tfidf_matrix.npy", tfidf_matrix)

# Save the TF-IDF vectorizer
with open("/content/drive/MyDrive/tfidf_vectorizer.pkl", "wb") as file:
    pickle.dump(tfidfModel, file)


In [ ]:
tfidf_df = pd.DataFrame(tfidfModel.fit_transform(df['text_lemmatized']).todense())
tfidf_df.columns = sorted(tfidfModel.vocabulary_)
tfidf_df

,aa,aal,aaland,aalto,aaltonen,aapl,aaron,aava,aazhang,ab,...,zu,zurich,zxx,àkersberga,àland,àlandsbanken,àmñl,ál,áá,äñnekoski
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5837,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5838,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5839,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5840,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Split the data

In [ ]:
x=tfidf_df
y=df[['Sentiment']]
y= np.ravel(y)

Model Build

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=42)

#SMOTE
smote = SMOTE(random_state=42)
x_train_resampled, y_train_resampled = smote.fit_resample(x_train, y_train)

In [ ]:
Classifiers = [
    SVC(kernel='linear',C=1.0),
    LogisticRegression(C=1.0,solver='lbfgs',max_iter=300),
    KNeighborsClassifier(n_neighbors=16,weights='uniform'),
    DecisionTreeClassifier(max_depth=25,criterion='gini', min_samples_split=250),
    RandomForestClassifier(n_estimators=300, criterion='gini'),
    AdaBoostClassifier(n_estimators=200,learning_rate=1.0),
    SGDClassifier(loss='modified_huber', penalty='elasticnet', alpha=0.001, l1_ratio=1e-05),
    XGBClassifier(n_estimators= 200,learning_rate=0.01,max_depth=4,gamma=0.1)
]

In [ ]:
model = LogisticRegression(C=1.0,solver='lbfgs',max_iter=300)
model = model.fit(x_train_resampled, y_train_resampled)
filename = '/content/drive/MyDrive/finalized_model.sav'
joblib.dump(model, filename)

['/content/drive/MyDrive/finalized_model.sav']

In [ ]:
from sklearn.calibration import CalibratedClassifierCV
Accuracy = []
Model = []
F1 = []
Loss = []

In [ ]:
for classifier in Classifiers:
    model = classifier.fit(x_train_resampled, y_train_resampled)
    calibrator = CalibratedClassifierCV(model, cv='prefit')
    modelx=calibrator.fit(x_train_resampled, y_train_resampled)
    pred = model.predict(x_test)
    pred_proba = modelx.predict_proba(x_test)
    #accuracy
    accuracy = accuracy_score(y_test, pred)*100
    # macro f1 score
    f1 = f1_score(y_test, pred,average='macro')*100
    #log loss
    loss = log_loss(y_pred = pred_proba, y_true=y_test)

    Loss.append(loss)
    F1.append(f1)
    Accuracy.append(accuracy)
    Model.append(classifier.__class__.__name__)

In [ ]:
result = pd.DataFrame({'Models':Model,'F1':F1, 'Loss':Loss})
result['Accuracy'] = Accuracy
result = result.sort_values(by='Accuracy',ascending=False)
result

,Models,F1,Loss,Accuracy
6,SGDClassifier,67.512857,0.821780,71.876783
1,LogisticRegression,66.973051,0.815321,71.591557
0,SVC,66.018201,1.181532,70.678836
5,AdaBoostClassifier,61.136058,0.939359,66.400456
4,RandomForestClassifier,55.079101,1.584246,64.917285
7,XGBClassifier,57.327240,0.861011,62.977752
3,DecisionTreeClassifier,55.257232,0.953702,59.783229
2,KNeighborsClassifier,30.573195,1.117000,30.861380


,Models,F1,Loss,Accuracy
0,SVC,61.652389,0.858977,72.047918
7,XGBClassifier,62.112637,0.750971,71.192242
1,LogisticRegression,57.491691,0.758589,70.279521
6,SGDClassifier,58.944010,0.703795,70.222476
5,AdaBoostClassifier,60.586524,0.911787,68.054763
4,RandomForestClassifier,51.344081,0.952408,67.427268
3,DecisionTreeClassifier,52.452618,0.857156,63.833428
2,KNeighborsClassifier,43.736605,0.894677,59.098688


Word-Embedding

In [ ]:
import pandas as pd
import gensim

In [ ]:
modelw = gensim.models.Word2Vec(
    window=5, # it takes 5 words before and 5 words ahead for training
    min_count=3, # consider at least 3 words as sentence
    workers=4, # cpu threads can be used
)

In [ ]:
modelw.build_vocab(list, progress_per=500) # Indicates how many words to process before showing/updating the progress

In [ ]:
modelw.save("/content/drive/MyDrive/word_embedding.bin")

In [ ]:
vocabulary_wm=modelw.wv.key_to_index
loveidx = modelw.wv.key_to_index["love"]
loveidx

1831

In [ ]:
modelw.save("finacial_word_embedding.model")
modelw

In [ ]:
# define default vector function
def default_vector():
    return np.zeros(100)

In [ ]:
vectors = [modelw.wv[word] if word in modelw.wv else default_vector() for word in df['text_lemmatized']]

In [ ]:
x=vectors
y=df[['Sentiment']]
y= np.ravel(y)

Model Build

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=20)

In [ ]:
Classifiers = [
    SVC(),
    LogisticRegression(C=0.000000001,solver='liblinear',max_iter=200),
    KNeighborsClassifier(n_neighbors=5, weights='uniform', p=1),
    DecisionTreeClassifier(),
    RandomForestClassifier(n_estimators=200),
    AdaBoostClassifier(),
    SGDClassifier(),
    XGBClassifier(learning_rate=0.01,max_depth=4,gamma=0.1)
]

In [ ]:
model = SVC()
model = model.fit(x_train, y_train)
filename = '/content/drive/MyDrive/finalized_model_WE.sav'
joblib.dump(model, filename)

['/content/drive/MyDrive/finalized_model_WE.sav']

In [ ]:
from sklearn.calibration import CalibratedClassifierCV
Accuracy = []
Model = []
F1 = []
Loss = []

In [ ]:
for classifier in Classifiers:
    model = classifier.fit(x_train, y_train)
    calibrator = CalibratedClassifierCV(model, cv='prefit')
    modelx=calibrator.fit(x_train, y_train)
    pred = model.predict(x_test)
    pred_proba = modelx.predict_proba(x_test)
    #accuracy
    accuracy = accuracy_score(pred, y_test)*100
    # macro f1 score
    f1 = f1_score(y_test, pred,average='macro')*100
    #log loss
    loss = log_loss(y_pred = pred_proba, y_true=y_test)

    Loss.append(loss)
    F1.append(f1)
    Accuracy.append(accuracy)
    Model.append(classifier.__class__.__name__)

In [ ]:
result = pd.DataFrame({'Models':Model,'F1':F1, 'Loss':Loss})
result['Accuracy'] = Accuracy
result = result.sort_values(by='Accuracy',ascending=False)
result

,Models,F1,Loss,Accuracy
0,SVC,23.221809,0.988610,53.451226
1,LogisticRegression,23.221809,0.988610,53.451226
2,KNeighborsClassifier,23.221809,0.988610,53.451226
3,DecisionTreeClassifier,23.221809,0.988609,53.451226
4,RandomForestClassifier,23.221809,0.988609,53.451226
5,AdaBoostClassifier,23.221809,0.988609,53.451226
6,SGDClassifier,23.221809,0.988624,53.451226
7,XGBClassifier,23.221809,0.988610,53.451226


Similar words

In [ ]:
modelw.wv.most_similar("idea")

[('give', 0.30274683237075806),
 ('establish', 0.3008919358253479),
 ('extensive', 0.2953038811683655),
 ('there', 0.2934221625328064),
 ('founder', 0.28292131423950195),
 ('ragutis', 0.28174591064453125),
 ('amazon', 0.27457502484321594),
 ('tlsn', 0.2732101082801819),
 ('productivity', 0.2696335017681122),
 ('door', 0.2687860131263733)]

In [ ]:
modelw.wv.most_similar("loss")

[('popular', 0.39171284437179565),
 ('bulk', 0.3328348398208618),
 ('lowest', 0.326032429933548),
 ('voima', 0.30996373295783997),
 ('court', 0.28797197341918945),
 ('morgan', 0.28105247020721436),
 ('marcel', 0.28103601932525635),
 ('nflx', 0.2779015302658081),
 ('ubs', 0.2775871455669403),
 ('combination', 0.27663454413414)]

In [ ]:
modelw.wv.most_similar("environment")

[('flexible', 0.37336939573287964),
 ('aer', 0.35727155208587646),
 ('research', 0.32602131366729736),
 ('backup', 0.32416099309921265),
 ('qcom', 0.3194943368434906),
 ('reduced', 0.315827339887619),
 ('yit', 0.31339186429977417),
 ('fouryear', 0.3086225092411041),
 ('stx', 0.3081100583076477),
 ('estonia', 0.3073159456253052)]

In [ ]:
modelw.wv.most_similar("agreement")

[('understand', 0.33902254700660706),
 ('stronger', 0.30997365713119507),
 ('operative', 0.30677172541618347),
 ('bln', 0.30514824390411377),
 ('connectivity', 0.30499759316444397),
 ('navteq', 0.3049728572368622),
 ('love', 0.2956956923007965),
 ('glass', 0.2752869129180908),
 ('asked', 0.2728716731071472),
 ('novartis', 0.26980528235435486)]

In [ ]:
modelw.wv.most_similar("comparable")

[('possibility', 0.3922279477119446),
 ('very', 0.34374096989631653),
 ('ragutis', 0.33894771337509155),
 ('strengthen', 0.33276766538619995),
 ('expand', 0.3118973672389984),
 ('pohjola', 0.3063986897468567),
 ('box', 0.30302029848098755),
 ('dc', 0.3015299141407013),
 ('estonia', 0.2965189218521118),
 ('yearonyear', 0.2919650375843048)]

BAG-OF-WORDS

In [ ]:
bow_model = CountVectorizer()
print(bow_model.fit_transform(var))

  (0, 8547)	1
  (0, 3377)	1
  (0, 8455)	2
  (0, 9388)	1
  (0, 4760)	1
  (0, 799)	1
  (0, 3463)	1
  (0, 7903)	1
  (0, 1152)	1
  (0, 6648)	1
  (0, 4866)	2
  (0, 711)	1
  (0, 7508)	1
  (0, 1576)	1
  (0, 6332)	1
  (0, 6991)	1
  (0, 5478)	1
  (0, 1732)	1
  (0, 353)	2
  (0, 5595)	1
  (0, 6449)	1
  (0, 1557)	1
  (0, 5369)	1
  (1, 2698)	1
  (1, 5857)	1
  :	:
  (5840, 3967)	1
  (5841, 353)	1
  (5841, 3158)	1
  (5841, 3245)	1
  (5841, 4261)	1
  (5841, 3546)	1
  (5841, 3703)	1
  (5841, 4947)	1
  (5841, 3203)	1
  (5841, 343)	1
  (5841, 7355)	1
  (5841, 8084)	2
  (5841, 827)	1
  (5841, 5926)	1
  (5841, 180)	1
  (5841, 1861)	1
  (5841, 3821)	1
  (5841, 5242)	1
  (5841, 9429)	1
  (5841, 4560)	1
  (5841, 6569)	1
  (5841, 4014)	1
  (5841, 4465)	1
  (5841, 4629)	1
  (5841, 8170)	1


In [ ]:
bow_model.vocabulary_

{'the': 8547,
 'geosolutions': 3377,
 'technology': 8455,
 'will': 9388,
 'leverage': 4760,
 'benefon': 799,
 'gps': 3463,
 'solution': 7903,
 'by': 1152,
 'providing': 6648,
 'location': 4866,
 'based': 711,
 'search': 7508,
 'community': 1576,
 'platform': 6332,
 'relevant': 6991,
 'multimedia': 5478,
 'content': 1732,
 'and': 353,
 'new': 5595,
 'powerful': 6449,
 'commercial': 1557,
 'model': 5369,
 'esi': 2698,
 'on': 5857,
 'low': 4928,
 'down': 2341,
 'to': 8663,
 'bk': 884,
 'real': 6849,
 'possibility': 6424,
 'for': 3158,
 'last': 4668,
 'quarter': 6729,
 'of': 5808,
 'componenta': 1627,
 'net': 5581,
 'sale': 7368,
 'doubled': 2334,
 'eurm': 2741,
 'from': 3245,
 'same': 7383,
 'period': 6201,
 'year': 9512,
 'earlier': 2423,
 'while': 9362,
 'it': 4261,
 'moved': 5445,
 'zero': 9564,
 'pretax': 6513,
 'profit': 6581,
 'loss': 4915,
 'according': 50,
 'finnishrussian': 3060,
 'chamber': 1344,
 'commerce': 1556,
 'all': 258,
 'major': 5029,
 'construction': 1714,
 'company': 

In [ ]:
bow = bow_model.fit_transform(var).todense()
np.save("/content/drive/MyDrive/bow_matrix.npy", bow)

# Save the TF-IDF vectorizer
with open("/content/drive/MyDrive/bow_vectorizer.pkl", "wb") as file:
    pickle.dump(bow_model, file)

In [ ]:
bow_df = pd.DataFrame(bow_model.fit_transform(var).todense())
bow_df.columns = sorted(bow_model.vocabulary_)
bow_df

,aa,aal,aaland,aalto,aaltonen,aapl,aaron,aava,aazhang,ab,...,zu,zurich,zxx,àkersberga,àland,àlandsbanken,àmñl,ál,áá,äñnekoski
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5837,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5838,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5839,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5840,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
#split the data into x and y
x=bow_df
y=df[['Sentiment']]
y=np.ravel(y)

Model Build

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=42)

In [ ]:
model = SVC(kernel='linear', C=0.1)
model = model.fit(x_train, y_train)
filename = '/content/drive/MyDrive/finalized_model_bow.sav'
joblib.dump(model, filename)

['/content/drive/MyDrive/finalized_model_bow.sav']

In [ ]:
Classifiers = [
    SVC(kernel='linear', C=0.1),
    LogisticRegression(C=0.1, solver='liblinear'),
    KNeighborsClassifier(n_neighbors=20, weights='uniform', p=2),
    DecisionTreeClassifier(criterion='entropy', max_depth=20, min_samples_split=100),
    RandomForestClassifier(n_estimators=200,max_depth=100, min_samples_split=60),
    AdaBoostClassifier(n_estimators=200, learning_rate=1.0),
    SGDClassifier(loss='log', penalty='elasticnet', alpha=0.001, l1_ratio=0.5),
    XGBClassifier(learning_rate=1, max_depth=2, n_estimators=100)
]

In [ ]:
from sklearn.calibration import CalibratedClassifierCV
Accuracy = []
Model = []
F1 = []
Loss = []

In [ ]:
for classifier in Classifiers:
    model = classifier.fit(x_train, y_train)
    calibrator = CalibratedClassifierCV(model, cv='prefit')
    modelx=calibrator.fit(x_train, y_train)
    pred = model.predict(x_test)
    pred_proba = modelx.predict_proba(x_test)
    #accuracy
    accuracy = accuracy_score(y_test, pred)*100
    # macro f1 score
    f1 = f1_score(y_test, pred,average='macro')*100
    #log loss
    loss = log_loss(y_pred = pred_proba, y_true=y_test)

    Loss.append(loss)
    F1.append(f1)
    Accuracy.append(accuracy)
    Model.append(classifier.__class__.__name__)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:163: FutureWarning: The loss 'log' was deprecated in v1.1 and will be removed in version 1.3. Use `loss='log_loss'` which is equivalent.
  warnings.warn(


In [ ]:
result = pd.DataFrame({'Models':Model,'F1':F1, 'Loss':Loss})
result['Accuracy'] = Accuracy
result = result.sort_values(by='Accuracy',ascending=False)
result

,Models,F1,Loss,Accuracy
0,SVC,61.652389,0.858977,72.047918
7,XGBClassifier,62.112637,0.750971,71.192242
1,LogisticRegression,57.491691,0.758589,70.279521
6,SGDClassifier,58.944010,0.703795,70.222476
5,AdaBoostClassifier,60.586524,0.911787,68.054763
4,RandomForestClassifier,51.344081,0.952408,67.427268
3,DecisionTreeClassifier,52.452618,0.857156,63.833428
2,KNeighborsClassifier,43.736605,0.894677,59.098688


##FinBERT

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 48.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 64.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 46.9 MB/s eta 0:00:00


In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import pipeline

x = df['text_rv_digits']
y = df['Sentiment']
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=20)
x_test_array = x_test.tolist()
finbert = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone',num_labels=3)
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone')

nlp = pipeline("sentiment-analysis", model=finbert, tokenizer=tokenizer)

sentences = x_test_array

results = nlp(sentences)
print(results)  #LABEL_1: neutral; LABEL_2: positive; LABEL_0: negative
predicted_labels = [item['label'].lower() for item in results]

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


[{'label': 'Positive', 'score': 0.7664666175842285}, {'label': 'Negative', 'score': 0.9999986886978149}, {'label': 'Neutral', 'score': 0.9998948574066162}, {'label': 'Neutral', 'score': 0.9999901056289673}, {'label': 'Neutral', 'score': 0.9994668364524841}, {'label': 'Neutral', 'score': 0.9977549910545349}, {'label': 'Positive', 'score': 1.0}, {'label': 'Positive', 'score': 0.9999995231628418}, {'label': 'Neutral', 'score': 0.9999971389770508}, {'label': 'Neutral', 'score': 0.9997308850288391}, {'label': 'Neutral', 'score': 0.9999990463256836}, {'label': 'Neutral', 'score': 0.999987006187439}, {'label': 'Neutral', 'score': 0.9999945163726807}, {'label': 'Neutral', 'score': 0.9999876022338867}, {'label': 'Neutral', 'score': 0.9998204112052917}, {'label': 'Neutral', 'score': 0.9991956353187561}, {'label': 'Positive', 'score': 0.9999254941940308}, {'label': 'Negative', 'score': 0.9998875856399536}, {'label': 'Positive', 'score': 0.9992116689682007}, {'label': 'Neutral', 'score': 0.9999139

In [ ]:
y_test = y_test.tolist()

def transform_array(array):
    transformed_array = []
    class_mapping = {'positive': 2, 'negative': 0, 'neutral': 1}

    for item in array:
        transformed_array.append(class_mapping[item])

    return transformed_array

predicted_labels = transform_array(predicted_labels)
accuracy = accuracy_score(y_test, predicted_labels)
f1 = f1_score(y_test, predicted_labels, average='macro')

print("accuracy = ", accuracy)
print("f1_maro = ", f1 )

accuracy =  0.713063320022818
f1_maro =  0.6794270330415628
